# Event Extraction Agent 테스트 (Production Level)

이벤트 추출 에이전트 (Level 1) 테스트 노트북

## 역할: "연출 감독" (Director)
- 누가, 어디서, 무엇을 했는가?
- 직접적인 묘사 대신 **참조(Reference)**와 **행동 묘사(Visual Scene)**에 집중

## 그래프 연결용 (Neo4j Edges)
- `participants`: Character 노드와 연결 (INVOLVES)
- `location_ref`: Setting 노드와 연결 (HAPPENS_AT)
- `prev_event_id`: 이전 이벤트와 연결 (NEXT)

In [ ]:
import sys, os, json, asyncio
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path: sys.path.insert(0, project_root)
from dotenv import load_dotenv
load_dotenv(os.path.join(project_root, '.env'))
print(f"Project root: {project_root}")

In [ ]:
SAMPLE_STORY = """
어두운 숲 속에서 서진은 검을 쥐고 있었다. 키가 크고 검은 머리카락을 가진 그는 어렸을 때부터 검술을 익혀온 뛰어난 검객이었다.
"우리가 이곳에서 그를 만날 수 있을까?" 서진의 옆에 서 있던 하나가 물었다.
하나는 마을에서 가장 현명한 치료사였고, 항상 차분하고 신중했다. 그녀는 하얀 치료사 로브를 입고 있었다.

갑자기 나무 사이에서 그림자가 나타났다. 그것은 이민호였다. 한때는 서진의 가장 친한 친구였지만,
지금은 그들의 마을을 배신한 적이 되어버렸다. 그의 차가운 눈빛에서 분노와 슬픔이 느껴졌다.

"오랜만이군, 서진." 이민호가 차갑게 말했다. 검은 갑옷을 입은 그는 예전보다 더 날카로워 보였다.
서진은 용감하게 앞으로 나섰다. 긴장감이 최고조에 달했다. "왜 마을을 배신했어, 민호?"

박서연이라는 젊은 여성이 숲에서 뛰어나왔다. 그녀는 마을의 정탐꾼으로, 빠르고 민첩했다.
"경비병들이 오고 있어요! 빨리 결정해야 해요!" 그녀의 목소리에는 급박함이 묻어났다.
"""

def create_base_state(story=SAMPLE_STORY):
    return {"content": story, "project_id": "test-001", "document_id": "doc-001", 
            "job_id": "job-001", "callback_url": "http://localhost:8080/api/callback",
            "extracted_events": [], "extracted_characters": [],
            "consistency_report": {}, "retry_count": 0, 
            "messages": [], "errors": []}

def run_async(coro):
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            import nest_asyncio; nest_asyncio.apply()
            return loop.run_until_complete(coro)
        return asyncio.run(coro)
    except: return asyncio.run(coro)

def pretty_print(title, data):
    print(f"\n{'='*70}")
    print(f"📋 {title}")
    print(f"{'='*70}")
    print(json.dumps(data, ensure_ascii=False, indent=2))

## 1. Event Extraction 테스트 (Production Level)

In [ ]:
from app.agents.extraction.event import event_extraction_node

async def test_event_extraction():
    print("📅 Event Extraction (Production Level) 테스트...")
    state = create_base_state()
    return await event_extraction_node(state)

result = run_async(test_event_extraction())
pretty_print("이벤트 추출 결과", result)

## 2. Event 분석 - Neo4j Edge 및 이미지 Scene

In [ ]:
events = result.get("extracted_events", [])

print("\n" + "="*70)
print("📊 Event 분석 (Production Level)")
print("="*70)

for event in events:
    print(f"\n📌 [{event.get('event_id')}] {event.get('narrative_summary', event.get('description', 'N/A'))}")
    print(f"   Type: {event.get('event_type')}")
    print(f"   Importance: {event.get('importance', 'N/A')}/10")
    
    # Graph Connections
    print(f"\n   🔗 Graph Connections (Neo4j):")
    print(f"      participants: {event.get('participants', [])}")
    print(f"      location_ref: {event.get('location_ref', 'N/A')}")
    print(f"      prev_event_id: {event.get('prev_event_id', 'N/A')}")
    
    # Visual Scene
    if event.get('visual_scene'):
        print(f"\n   🎬 Visual Scene (이미지 생성용):")
        print(f"      \"{event.get('visual_scene')}\"")
        if event.get('camera_angle'):
            print(f"      Camera: {event.get('camera_angle')}")

## 3. 캐릭터-이벤트 연결 분석

In [ ]:
# Production Level 캐릭터 데이터 (character.py에서 추출된 형식)
production_characters = [
    {
        "name": "서진",
        "role": "protagonist",
        "visual": {
            "appearance": ["tall", "dark hair"],
            "attire": ["holding sword"]
        },
        "personality": {
            "core_traits": ["brave", "skilled swordsman"],
            "flaws": ["impulsive"]
        },
        "relationships": [
            {"target": "이민호", "type": "ENEMY", "history": "former_friend", "strength": 8}
        ]
    },
    {
        "name": "이민호",
        "role": "antagonist",
        "visual": {
            "appearance": ["cold eyes"],
            "attire": ["wearing black armor"]
        },
        "personality": {
            "core_traits": ["cold", "skilled", "betrayer"],
            "flaws": ["bitter", "vengeful"]
        },
        "relationships": [
            {"target": "서진", "type": "ENEMY", "history": "former_friend", "strength": 8}
        ]
    }
]

print("\n" + "="*70)
print("🔗 캐릭터-이벤트 연결 분석")
print("="*70)

# 캐릭터별 이벤트 참여도
char_events = {}
for event in events:
    for participant in event.get("participants", []):
        if participant not in char_events:
            char_events[participant] = []
        char_events[participant].append(event.get("event_id"))

for char in production_characters:
    name = char.get("name")
    print(f"\n🧑 {name} ({char.get('role')})")
    print(f"   참여 이벤트: {char_events.get(name, [])}")
    
    # 관계 기반 분석
    for rel in char.get("relationships", []):
        target = rel.get("target")
        shared_events = set(char_events.get(name, [])) & set(char_events.get(target, []))
        if shared_events:
            print(f"   🔗 {target}와 공유 이벤트: {list(shared_events)} ({rel.get('type')}, was: {rel.get('history')})")

## 4. 타임라인 시각화 (prev_event_id 체인)

In [ ]:
print("\n" + "="*70)
print("📅 이벤트 타임라인")
print("="*70)

# 이벤트를 순서대로 표시
for i, event in enumerate(events, 1):
    importance = event.get('importance', 5)
    bar = '█' * importance + '░' * (10 - importance)
    event_type_emoji = {
        'action': '⚔️',
        'dialogue': '💬',
        'revelation': '💡',
        'flashback': '⏪',
        'confrontation': '⚡'
    }.get(event.get('event_type', ''), '📌')
    
    # Timeline chain
    prev = event.get('prev_event_id', '')
    chain_indicator = f"→ " if prev else "● "
    
    print(f"\n{chain_indicator}{event_type_emoji} [{event.get('event_id')}] {event.get('narrative_summary', event.get('description', 'N/A'))[:50]}...")
    print(f"   Importance: [{bar}] {importance}/10")
    print(f"   Participants: {', '.join(event.get('participants', []))}")
    print(f"   Location: {event.get('location_ref', 'Unknown')}")
    if prev:
        print(f"   Timeline: {prev} → {event.get('event_id')}")

## 5. Re-extraction 테스트 (충돌 해결)

In [ ]:
async def test_re_extraction():
    print("🔄 Event Re-extraction 테스트...")
    state = create_base_state()
    state["retry_count"] = 1
    
    # 이전 추출 결과
    state["extracted_events"] = [
        {
            "event_id": "E001", 
            "narrative_summary": "서진이 숲에서 대기",
            "participants": ["서진"],
            "location_ref": "Dark Forest",
            "event_type": "action",
            "visual_scene": "A man holding a sword",
            "importance": 5
        },
        {
            "event_id": "E002", 
            "narrative_summary": "서진과 이민호의 대화",
            "participants": ["서진", "이민호"],
            "location_ref": "Dark Forest",
            "prev_event_id": "E001",
            "event_type": "action",  # 잘못됨 - confrontation 또는 dialogue여야 함
            "visual_scene": "Two men talking",
            "importance": 5  # 잘못됨 - 중요도 더 높아야 함
        }
    ]
    
    # 충돌 정보
    state["consistency_report"] = {
        "conflicts": [
            {
                "type": "EVENT_TYPE_CONFLICT",
                "description": "E002는 대립 장면이므로 event_type이 'confrontation' 또는 'dialogue'여야 함",
                "severity": "MEDIUM",
                "affected_elements": ["E002"]
            },
            {
                "type": "IMPORTANCE_CONFLICT", 
                "description": "E002는 스토리의 핵심 갈등을 드러내는 장면이므로 importance가 8 이상이어야 함",
                "severity": "LOW",
                "affected_elements": ["E002"]
            }
        ]
    }
    
    return await event_extraction_node(state)

re_result = run_async(test_re_extraction())
pretty_print("Re-extraction 결과", re_result)

In [ ]:
# Re-extraction 분석
print("\n" + "="*70)
print("📊 Re-extraction 분석")
print("="*70)

re_events = re_result.get("extracted_events", [])

for event in re_events:
    event_id = event.get('event_id')
    if '대화' in event.get('narrative_summary', '') or '대립' in event.get('narrative_summary', '') or event_id == 'E002':
        print(f"\n📌 [{event_id}] {event.get('narrative_summary', event.get('description', 'N/A'))}")
        
        is_correct_type = event.get('event_type') in ['dialogue', 'confrontation']
        is_correct_importance = event.get('importance', 0) >= 8
        
        print(f"   Type: {event.get('event_type')} {'✅' if is_correct_type else '❌'}")
        print(f"   Importance: {event.get('importance')}/10 {'✅' if is_correct_importance else '❌'}")